In [136]:
!pip install vecstack

In [0]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
#from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from collections import Counter #for Smote, 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import warnings
warnings.filterwarnings("ignore")

In [138]:
from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive

%cd /gdrive


trainfile = r'/gdrive/My Drive/CIS508/Assignment-6/Churn-Train-1.csv'
train_data = pd.read_csv(trainfile)

testfile = r'/gdrive/My Drive/CIS508/Assignment-6/Churn-Test-1.csv'
test_data = pd.read_csv(testfile)

print(train_data.shape)
print(train_data.dtypes)
train_data.head()

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive
(1165, 21)
State              object
Account Length      int64
Area Code           int64
Phone              object
Int'l Plan         object
VMail Plan         object
VMail Message       int64
Day Mins          float64
Day Calls           int64
Day Charge        float64
Eve Mins          float64
Eve Calls           int64
Eve Charge        float64
Night Mins        float64
Night Calls         int64
Night Charge      float64
Intl Mins         float64
Intl Calls          int64
Intl Charge       float64
CustServ Calls      int64
TARGET              int64
dtype: object


,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,TARGET
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0


In [139]:
print(test_data.shape)
print(test_data.dtypes)
test_data.head()

(1262, 21)
State              object
Account Length      int64
Area Code           int64
Phone              object
Int'l Plan         object
VMail Plan         object
VMail Message       int64
Day Mins          float64
Day Calls           int64
Day Charge        float64
Eve Mins          float64
Eve Calls           int64
Eve Charge        float64
Night Mins        float64
Night Calls       float64
Night Charge      float64
Intl Mins         float64
Intl Calls        float64
Intl Charge       float64
CustServ Calls    float64
TARGET            float64
dtype: object


,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,TARGET
0,VT,50,415,387-5891,yes,yes,26,307.1,94,52.21,289.4,78,24.60,174.9,109.0,7.87,8.0,3.0,2.16,0.0,0.0
1,UT,72,415,368-8026,no,no,0,118.2,106,20.09,167.2,136,14.21,214.2,106.0,9.64,12.2,3.0,3.29,3.0,0.0
2,KS,130,510,332-9446,no,no,0,154.0,95,26.18,205.9,106,17.50,233.7,75.0,10.52,12.9,1.0,3.48,1.0,0.0
3,NV,143,408,393-5284,no,no,0,155.5,101,26.44,213.4,89,18.14,237.9,61.0,10.71,7.6,11.0,2.05,1.0,0.0
4,DE,89,510,376-1677,yes,no,0,125.6,108,21.35,213.0,90,18.11,181.7,108.0,8.18,5.4,5.0,1.46,1.0,0.0


In [140]:
#DROP COLUMNS WITH LOTS OF MISSING VALUES===============================
#CAN ALSO DROP ROWS WITH LOTS OF MISSING VALUES
#Combine Train data and test data first so that the SAME COLUMNS are DROPPED in each
combined_Data = pd.concat([train_data, test_data], keys=[0,1])
percent=int(0.6*(combined_Data.shape[0]))
print(percent)
#Drop columns that have less than "thresh" number of non_Nans
Processing = combined_Data.dropna(thresh=percent,axis=1)
print(Processing.shape)

1456
(2427, 21)


In [141]:
#NOW IMPUTE MISSING VALUES FOR THE OTHER COLUMNS=========================
#IMPUTE (SUBSTITUTE) MEAN VALUES FOR NaN IN NUMERIC COLUMNS 
numeric = Processing.select_dtypes(include=['int','float64']).columns
for num in numeric:
  Processing[num] = Processing[num].fillna(Processing[num].mean())

#IMPUTE (SUBSTITUTE) MODE VALUES FOR NaN IN CATEGORICAL COLUMNS
train_cat_cols = Processing.select_dtypes(exclude=['int','float64']).columns#selecting the categorical columns
for colss in train_cat_cols:
  Processing[colss] = Processing[colss].fillna(Processing[colss].mode()[0])

print(Processing.head())

    State  Account Length  Area Code  ... Intl Charge CustServ Calls TARGET
0 0    KS             128        415  ...        2.70            1.0    0.0
  1    OH             107        415  ...        3.70            1.0    0.0
  2    NJ             137        415  ...        3.29            0.0    0.0
  3    OH              84        408  ...        1.78            2.0    0.0
  4    OK              75        415  ...        2.73            3.0    0.0

[5 rows x 21 columns]


In [142]:
#Check final state of nulls in the columns
Processing.isnull().sum()

State             0
Account Length    0
Area Code         0
Phone             0
Int'l Plan        0
VMail Plan        0
VMail Message     0
Day Mins          0
Day Calls         0
Day Charge        0
Eve Mins          0
Eve Calls         0
Eve Charge        0
Night Mins        0
Night Calls       0
Night Charge      0
Intl Mins         0
Intl Calls        0
Intl Charge       0
CustServ Calls    0
TARGET            0
dtype: int64

In [143]:
#dropping unnecessary columns
Processing_F=Processing.drop(columns=['Phone'])
print(Processing_F.shape)

(2427, 20)


In [0]:
#Export Processed data 
F_Train = Processing_F.xs(0)
F_Test = Processing_F.xs(1)

#
combine = [F_Train, F_Test]
F_Train.shape, F_Test.shape
for dataset in combine:
  dataset["Int'l Plan"] = dataset["Int'l Plan"].map({'yes': 1, 'no': 0}).astype(int)
  dataset['VMail Plan'] = dataset['VMail Plan'].map({'yes': 1, 'no': 0}).astype(int)


F_Train.to_csv(r'/gdrive/My Drive/CIS508/Assignment-6/Preprocess_Train.csv',index=None)
F_Test.to_csv(r'/gdrive/My Drive/CIS508/Assignment-6/Preprocess_Test.csv',index=None)

In [145]:
F_Train.head()

,State,Account Length,Area Code,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,TARGET
0,KS,128,415,0,1,25,265.1,110,45.07,197.4,99,16.78,244.7,91.0,11.01,10.0,3.0,2.70,1.0,0.0
1,OH,107,415,0,1,26,161.6,123,27.47,195.5,103,16.62,254.4,103.0,11.45,13.7,3.0,3.70,1.0,0.0
2,NJ,137,415,0,0,0,243.4,114,41.38,121.2,110,10.30,162.6,104.0,7.32,12.2,5.0,3.29,0.0,0.0
3,OH,84,408,1,0,0,299.4,71,50.90,61.9,88,5.26,196.9,89.0,8.86,6.6,7.0,1.78,2.0,0.0
4,OK,75,415,1,0,0,166.7,113,28.34,148.3,122,12.61,186.9,121.0,8.41,10.1,3.0,2.73,3.0,0.0


In [146]:
F_Test.head()

,State,Account Length,Area Code,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,TARGET
0,VT,50,415,1,1,26,307.1,94,52.21,289.4,78,24.60,174.9,109.0,7.87,8.0,3.0,2.16,0.0,0.0
1,UT,72,415,0,0,0,118.2,106,20.09,167.2,136,14.21,214.2,106.0,9.64,12.2,3.0,3.29,3.0,0.0
2,KS,130,510,0,0,0,154.0,95,26.18,205.9,106,17.50,233.7,75.0,10.52,12.9,1.0,3.48,1.0,0.0
3,NV,143,408,0,0,0,155.5,101,26.44,213.4,89,18.14,237.9,61.0,10.71,7.6,11.0,2.05,1.0,0.0
4,DE,89,510,1,0,0,125.6,108,21.35,213.0,90,18.11,181.7,108.0,8.18,5.4,5.0,1.46,1.0,0.0


In [147]:
#Import Processed data and check it
trainfile = r'/gdrive/My Drive/CIS508/Assignment-6/Preprocess_Train.csv'
train_data = pd.read_csv(trainfile)

testfile = r'/gdrive/My Drive/CIS508/Assignment-6/Preprocess_Test.csv'
test_data = pd.read_csv(testfile)

print(train_data.shape)
print(test_data.shape)

(1165, 20)
(1262, 20)


In [148]:
train_data.describe()

,Account Length,Area Code,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,TARGET
count,1165.000000,1165.000000,1165.000000,1165.000000,1165.000000,1165.000000,1165.000000,1165.000000,1165.000000,1165.000000,1165.000000,1165.000000,1165.000000,1165.000000,1165.000000,1165.000000,1165.000000,1165.000000,1165.000000
mean,100.107296,437.899571,0.098712,0.284120,8.319313,181.588412,100.605150,30.870532,199.351073,99.871245,16.945021,199.931159,100.104721,8.996927,10.228155,4.550215,2.762172,1.539056,0.125322
std,40.128933,42.861654,0.298404,0.451188,13.730382,54.392831,19.819381,9.246717,52.633766,20.077314,4.473876,51.233628,19.899559,2.305449,2.833913,2.538757,0.765126,1.307352,0.331226
min,1.000000,408.000000,0.000000,0.000000,0.000000,12.500000,30.000000,2.130000,31.200000,12.000000,2.650000,43.700000,42.000000,1.970000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,73.000000,408.000000,0.000000,0.000000,0.000000,146.800000,88.000000,24.960000,164.400000,87.000000,13.970000,167.000000,87.000000,7.520000,8.400000,3.000000,2.270000,1.000000,0.000000
50%,98.000000,415.000000,0.000000,0.000000,0.000000,182.300000,101.000000,30.990000,200.200000,100.000000,17.020000,200.500000,101.000000,9.020000,10.300000,4.000000,2.780000,1.000000,0.000000
75%,128.000000,510.000000,0.000000,1.000000,21.000000,216.000000,114.000000,36.720000,234.100000,114.000000,19.900000,236.600000,114.000000,10.650000,12.100000,6.000000,3.270000,2.000000,0.000000
max,243.000000,510.000000,1.000000,1.000000,51.000000,350.800000,165.000000,59.640000,351.600000,168.000000,29.890000,364.300000,175.000000,16.390000,20.000000,19.000000,5.400000,9.000000,1.000000


In [149]:
train_data.head()

,State,Account Length,Area Code,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,TARGET
0,KS,128,415,0,1,25,265.1,110,45.07,197.4,99,16.78,244.7,91.0,11.01,10.0,3.0,2.70,1.0,0.0
1,OH,107,415,0,1,26,161.6,123,27.47,195.5,103,16.62,254.4,103.0,11.45,13.7,3.0,3.70,1.0,0.0
2,NJ,137,415,0,0,0,243.4,114,41.38,121.2,110,10.30,162.6,104.0,7.32,12.2,5.0,3.29,0.0,0.0
3,OH,84,408,1,0,0,299.4,71,50.90,61.9,88,5.26,196.9,89.0,8.86,6.6,7.0,1.78,2.0,0.0
4,OK,75,415,1,0,0,166.7,113,28.34,148.3,122,12.61,186.9,121.0,8.41,10.1,3.0,2.73,3.0,0.0


In [150]:
#Copy Train data excluding ID column
train_data_Copy = train_data.iloc[:,:-1].copy()
#Copy Test data excluding ID column 
test_data_Copy = test_data.iloc[:,:-1].copy()

print(train_data_Copy.head())
print(test_data_Copy.head())

  State  Account Length  Area Code  ...  Intl Calls  Intl Charge  CustServ Calls
0    KS             128        415  ...         3.0         2.70             1.0
1    OH             107        415  ...         3.0         3.70             1.0
2    NJ             137        415  ...         5.0         3.29             0.0
3    OH              84        408  ...         7.0         1.78             2.0
4    OK              75        415  ...         3.0         2.73             3.0

[5 rows x 19 columns]
  State  Account Length  Area Code  ...  Intl Calls  Intl Charge  CustServ Calls
0    VT              50        415  ...         3.0         2.16             0.0
1    UT              72        415  ...         3.0         3.29             3.0
2    KS             130        510  ...         1.0         3.48             1.0
3    NV             143        408  ...        11.0         2.05             1.0
4    DE              89        510  ...         5.0         1.46             1.0

[5 r

In [151]:
#List of Categorical Features
train_cat_cols = train_data_Copy.select_dtypes(exclude=['float','int']).columns
print(train_cat_cols.shape)
print(train_cat_cols)

#Combine Train and test for one Hot Encoding
combined_Data = pd.concat([train_data_Copy, test_data_Copy], keys=[2,3])
print(combined_Data.shape)

#Do one Hot encoding for categorical features
combined_Data = pd.get_dummies(combined_Data,columns=train_cat_cols)
print(combined_Data.shape)

#save feature names because we are going to do rescaler
combined_Data_col = combined_Data.columns

#Separate Train data and test data
Xtrain = combined_Data.xs(2)
Xtest = combined_Data.xs(3)

#Normalize for train data
Xscaler = StandardScaler()
Xscaled = Xscaler.fit(Xtrain)
XscalerTrans = Xscaler.transform(Xtrain)
#give feature names back which we saved before
Xtrain = pd.DataFrame(XscalerTrans, columns=combined_Data_col)
print(Xtrain.head())
print(Xtrain.shape)

#Normalize for test data
Xscaled = Xscaler.fit(Xtest)
XscalerTrans = Xscaler.transform(Xtest)
#give feature names back which we saved before
Xtest = pd.DataFrame(XscalerTrans, columns=combined_Data_col)

(1,)
Index(['State'], dtype='object')
(2427, 19)
(2427, 69)
   Account Length  Area Code  Int'l Plan  ...  State_WI  State_WV  State_WY
0        0.695376  -0.534497   -0.330944  ... -0.145032 -0.165339 -0.159775
1        0.171838  -0.534497   -0.330944  ... -0.145032 -0.165339 -0.159775
2        0.919749  -0.534497   -0.330944  ... -0.145032 -0.165339 -0.159775
3       -0.401561  -0.697883    3.021661  ... -0.145032 -0.165339 -0.159775
4       -0.625934  -0.534497    3.021661  ... -0.145032 -0.165339 -0.159775

[5 rows x 69 columns]
(1165, 69)


In [152]:
#
print(Xtrain.shape)
print(Xtest.shape)

print(Xtrain.head)
print(Xtest.head)

(1165, 69)
(1262, 69)
<bound method NDFrame.head of       Account Length  Area Code  Int'l Plan  ...  State_WI  State_WV  State_WY
0           0.695376  -0.534497   -0.330944  ... -0.145032 -0.165339 -0.159775
1           0.171838  -0.534497   -0.330944  ... -0.145032 -0.165339 -0.159775
2           0.919749  -0.534497   -0.330944  ... -0.145032 -0.165339 -0.159775
3          -0.401561  -0.697883    3.021661  ... -0.145032 -0.165339 -0.159775
4          -0.625934  -0.534497    3.021661  ... -0.145032 -0.165339 -0.159775
...              ...        ...         ...  ...       ...       ...       ...
1160       -1.323985  -0.697883   -0.330944  ... -0.145032 -0.165339 -0.159775
1161       -1.498498   1.682889    3.021661  ... -0.145032 -0.165339 -0.159775
1162        1.817243   1.682889   -0.330944  ... -0.145032 -0.165339 -0.159775
1163        1.418357   1.682889   -0.330944  ... -0.145032 -0.165339 -0.159775
1164       -0.850308  -0.697883   -0.330944  ... -0.145032 -0.165339 -0.159775


In [153]:
#Select Target column
Ytrain = train_data["TARGET"]
print(Ytrain.shape)
print(Ytrain.head())

Ytest = test_data["TARGET"]
print(Ytest.shape)
print(Ytest.head())

(1165,)
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: TARGET, dtype: float64
(1262,)
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: TARGET, dtype: float64


In [154]:
#SMOTE==============================================================================
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(Ytrain))#before smote
sm = SMOTE(sampling_strategy='float', ratio=0.5)#no of minority /majority
X_res, Y_res = sm.fit_resample(Xtrain, Ytrain)#after smote
print('Resampled dataset shape %s' % Counter(Y_res))#after smote
#new training set with x n y numbers changed #we can do tjis before we do all the previous model building

___________________________________________________________________
SMOTE

Original dataset shape Counter({0.0: 1019, 1.0: 146})
Resampled dataset shape Counter({0.0: 1019, 1.0: 509})


In [155]:

#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY ==================
clf = DecisionTreeClassifier()
clf.fit(X_res, Y_res)
clf_predict_train=clf.predict(X_res)


print("accuracy Score (training) for Decision TreE:{0:6f}".format(clf.score(X_res,Y_res)))
print("Confusion Matrix for Decision Tree")
print(confusion_matrix(Y_res,clf_predict_train))
print("Classification report for Decision Tree")
print(classification_report(Y_res,clf_predict_train))

#run cross-validation on best hyperparameters, get auc score
clf_cv_score = cross_val_score(clf, X_res, Y_res, cv=8, scoring="roc_auc")
print("=== All AUC Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Decision Tree: ",clf_cv_score.mean())


accuracy Score (training) for Decision TreE:1.000000
Confusion Matrix for Decision Tree
[[1019    0]
 [   0  509]]
Classification report for Decision Tree
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      1019
         1.0       1.00      1.00      1.00       509

    accuracy                           1.00      1528
   macro avg       1.00      1.00      1.00      1528
weighted avg       1.00      1.00      1.00      1528

=== All AUC Scores ===
[0.8828125  0.87109375 0.8984375  0.9175689  0.93719242 0.87351581
 0.94475691 0.93288339]


=== Mean AUC Score ===
Mean AUC Score - Decision Tree:  0.9072826467980565


In [156]:

Ypred_test = pd.DataFrame(clf.predict(Xtest),columns=["TARGET"])

Ypred_test.head()

Ypred_prob=pd.DataFrame(clf.predict_proba(Xtest))
Ypred_prob.head()


,0,1
0,0.0,1.0
1,0.0,1.0
2,1.0,0.0
3,1.0,0.0
4,0.0,1.0


In [157]:
#Hyperparameter tuning done for decision tree classifier
parameters={'min_samples_split' : range(10,100,10),'min_samples_leaf': range(2,8,1)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=10)
clf_random.fit(X_res, Y_res)
rand_parm=clf_random.best_params_
print(rand_parm)

#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf = DecisionTreeClassifier(**rand_parm)
clf.fit(X_res,Y_res)
clf_predict_train = clf.predict(X_res)

#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (training) after hypertuning for Decision Tree:{0:6f}".format(clf.score(X_res,Y_res)))
print("Confusion Matrix after hypertuning for Decision Tree")
print(confusion_matrix(Y_res,clf_predict_train))
print("=== Classification Report after hypertuning for Decision Tree===")
print(classification_report(Y_res,clf_predict_train))

#run cross-validation on best hyperparameters, get auc score
clf_cv_score = cross_val_score(clf, X_res, Y_res, cv=8, scoring="roc_auc")
print("=== All AUC Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Decision Tree: ",clf_cv_score.mean())

{'min_samples_split': 60, 'min_samples_leaf': 4}
accuracy Score (training) after hypertuning for Decision Tree:0.923429
Confusion Matrix after hypertuning for Decision Tree
[[1001   18]
 [  99  410]]
=== Classification Report after hypertuning for Decision Tree===
              precision    recall  f1-score   support

         0.0       0.91      0.98      0.94      1019
         1.0       0.96      0.81      0.88       509

    accuracy                           0.92      1528
   macro avg       0.93      0.89      0.91      1528
weighted avg       0.93      0.92      0.92      1528

=== All AUC Scores ===
[0.90771484 0.91448975 0.940979   0.96911909 0.97182579 0.88782652
 0.96575428 0.94313211]


=== Mean AUC Score ===
Mean AUC Score - Decision Tree:  0.9376051733157379


In [158]:
Ypred_test = pd.DataFrame(clf.predict(Xtest),columns=["TARGET"])

Ypred_test.head()

Ypred_prob=pd.DataFrame(clf.predict_proba(Xtest))
Ypred_prob.head()


,0,1
0,0.0,1.0
1,0.0,1.0
2,1.0,0.0
3,1.0,0.0
4,0.0,1.0


In [159]:
#Random Forest =============================================================
#Default mode
rfc = RandomForestClassifier()
rfc.fit(X_res, Y_res)
rfc_predict_train=rfc.predict(X_res)
print("accuracy Score (training) for RandomForest:{0:6f}".format(rfc.score(X_res,Y_res)))
print("Confusion Matrix for Random Forest:")
print(confusion_matrix(Y_res,rfc_predict_train))
print("classification report for Random Forest:")
print(classification_report(Y_res,rfc_predict_train))
#run cross-validation on best parameters, get auc score
rfc_cv_score = cross_val_score(rfc, X_res, Y_res, cv=8, scoring="roc_auc")
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ",rfc_cv_score.mean())


accuracy Score (training) for RandomForest:0.997382
Confusion Matrix for Random Forest:
[[1018    1]
 [   3  506]]
classification report for Random Forest:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      1019
         1.0       1.00      0.99      1.00       509

    accuracy                           1.00      1528
   macro avg       1.00      1.00      1.00      1528
weighted avg       1.00      1.00      1.00      1528

=== All AUC Scores ===
[0.91271973 0.95318604 0.95013428 0.98972687 0.99846211 0.95213098
 0.97094113 0.97956505]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.9633582732242698


In [160]:
Ypred_test = pd.DataFrame(rfc.predict(Xtest),columns=["TARGET"])

Ypred_test.head()

Ypred_prob=pd.DataFrame(rfc.predict_proba(Xtest))
Ypred_prob.head()


,0,1
0,0.5,0.5
1,0.9,0.1
2,1.0,0.0
3,1.0,0.0
4,0.8,0.2


In [161]:
#Hyperparameter tuning for random forest
parameters={ 'n_estimators': range(50,100,10),'min_samples_split' : range(10,100,10),'min_samples_leaf': range(2,8,1)}
rfc_random = RandomizedSearchCV(rfc,parameters,n_iter=10)
rfc_random.fit(X_res, Y_res)
rand_parm_rfc=rfc_random.best_params_
print(rand_parm_rfc)

#contruct random forest using the best parameters
rfc= RandomForestClassifier(**rand_parm_rfc)
rfc.fit(X_res,Y_res)
rfc_predict_train = rfc.predict(X_res)
print("accuracy Score (training) after hypertuning for Random Forest:{0:6f}".format(rfc.score(X_res,Y_res)))
print("Confusion Matrix after hypertuning for Random Forest:")
print(confusion_matrix(Y_res,rfc_predict_train))
print("=== Classification Report after hypertuning for Random Forest:===")
print(classification_report(Y_res,rfc_predict_train))


#run cross-validation on best parameters, get auc score
rfc_cv_score = cross_val_score(rfc, X_res, Y_res, cv=8, scoring="roc_auc")
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ",rfc_cv_score.mean())


{'n_estimators': 80, 'min_samples_split': 30, 'min_samples_leaf': 2}
accuracy Score (training) after hypertuning for Random Forest:0.967932
Confusion Matrix after hypertuning for Random Forest:
[[1013    6]
 [  43  466]]
=== Classification Report after hypertuning for Random Forest:===
              precision    recall  f1-score   support

         0.0       0.96      0.99      0.98      1019
         1.0       0.99      0.92      0.95       509

    accuracy                           0.97      1528
   macro avg       0.97      0.95      0.96      1528
weighted avg       0.97      0.97      0.97      1528

=== All AUC Scores ===
[0.92028809 0.94897461 0.95751953 0.9886811  0.98966535 0.95063117
 0.97950256 0.97612798]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.963923800067594


In [162]:
Ypred_test = pd.DataFrame(rfc.predict(Xtest),columns=["TARGET"])

Ypred_test.head()

Ypred_prob=pd.DataFrame(rfc.predict_proba(Xtest))
Ypred_prob.head()

,0,1
0,0.420660,0.579340
1,0.870682,0.129318
2,0.835440,0.164560
3,0.889645,0.110355
4,0.656606,0.343394


In [163]:
#Neural networks=============================================================
#Default mode
nn = MLPClassifier()
nn.fit(X_res, Y_res)
nn_predict_train=nn.predict(X_res)
print("accuracy Score (training) for Neural network:{0:6f}".format(nn.score(X_res,Y_res)))
print("Confusion Matrix for neural network:")
print(confusion_matrix(Y_res,nn_predict_train))
print("=== Classification Report for neural network:===")
print(classification_report(Y_res,nn_predict_train))

#run cross-validation on best parameters, get auc score
nn_cv_score = cross_val_score(nn, X_res, Y_res, cv=8, scoring="roc_auc")
print("=== All AUC Scores ===")
print(nn_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Neural network: ",nn_cv_score.mean())

accuracy Score (training) for Neural network:1.000000
Confusion Matrix for neural network:
[[1019    0]
 [   0  509]]
=== Classification Report for neural network:===
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      1019
         1.0       1.00      1.00      1.00       509

    accuracy                           1.00      1528
   macro avg       1.00      1.00      1.00      1528
weighted avg       1.00      1.00      1.00      1528

=== All AUC Scores ===
[0.93408203 0.96105957 0.97290039 0.99360236 0.9800689  0.98537683
 0.99625047 0.99575053]


=== Mean AUC Score ===
Mean AUC Score - Neural network:  0.9773863849751936


In [164]:
Ypred_test = pd.DataFrame(nn.predict(Xtest),columns=["TARGET"])

Ypred_test.head()

Ypred_prob=pd.DataFrame(nn.predict_proba(Xtest))
Ypred_prob.head()

,0,1
0,0.001021,0.998979
1,0.910100,0.089900
2,0.991182,0.008818
3,0.999970,0.000030
4,0.984089,0.015911


In [165]:
 #Hyperparameter tuning for neural network
parameters={ 'activation': ['relu','logistic'], 'max_iter': range(100,150,10)}
nn_random = RandomizedSearchCV(nn,parameters,n_iter=10)
nn_random.fit(X_res, Y_res)
rand_parm_nn=nn_random.best_params_
print(rand_parm_nn)

#contruct mlclassifier using the best parameters
nn = MLPClassifier(**rand_parm_nn)
nn.fit(X_res, Y_res)
nn_predict_train = nn.predict(X_res)
print("accuracy Score (training) after hypertuning for Neural networks:{0:6f}".format(nn.score(X_res, Y_res)))
print("Confusion Matrix after hypertuning for Neural network:")
print(confusion_matrix(Y_res,nn_predict_train))
print("=== Classification Report after hypertuning for Neural network:===")
print(classification_report(Y_res,nn_predict_train))

#run cross-validation on best parameters, get auc score
nn_cv_score = cross_val_score(nn, X_res, Y_res, cv=8, scoring="roc_auc")
print("=== All AUC Scores ===")
print(nn_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Neural network: ",nn_cv_score.mean())
 

{'max_iter': 140, 'activation': 'relu'}
accuracy Score (training) after hypertuning for Neural networks:1.000000
Confusion Matrix after hypertuning for Neural network:
[[1019    0]
 [   0  509]]
=== Classification Report after hypertuning for Neural network:===
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      1019
         1.0       1.00      1.00      1.00       509

    accuracy                           1.00      1528
   macro avg       1.00      1.00      1.00      1528
weighted avg       1.00      1.00      1.00      1528

=== All AUC Scores ===
[0.94030762 0.95507812 0.96582031 0.99200295 0.97883858 0.98450194
 0.99787527 0.99637545]


=== Mean AUC Score ===
Mean AUC Score - Neural network:  0.9763500307548226


In [166]:
Ypred_test = pd.DataFrame(nn.predict(Xtest),columns=["TARGET"])

Ypred_test.head()

Ypred_prob=pd.DataFrame(nn.predict_proba(Xtest))
Ypred_prob.head()


,0,1
0,0.000604,0.999396
1,0.203483,0.796517
2,0.975153,0.024847
3,0.999968,0.000032
4,0.731876,0.268124


In [167]:
#SVC=============================================================
#Default mode
svc = SVC()
svc.fit(X_res, Y_res)
svc_predict_train=svc.predict(X_res)
print("accuracy Score (training) for svc:{0:6f}".format(svc.score(X_res, Y_res)))
print("Confusion Matrix for svc:")
print(confusion_matrix(Y_res,svc_predict_train))
print("=== Classification Report for svc:===")
print(classification_report(Y_res,svc_predict_train))

#run cross-validation on best parameters, get auc score
svc_cv_score = cross_val_score(svc, X_res, Y_res, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(svc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - svc: ",svc_cv_score.mean())

accuracy Score (training) for svc:0.947644
Confusion Matrix for svc:
[[979  40]
 [ 40 469]]
=== Classification Report for svc:===
              precision    recall  f1-score   support

         0.0       0.96      0.96      0.96      1019
         1.0       0.92      0.92      0.92       509

    accuracy                           0.95      1528
   macro avg       0.94      0.94      0.94      1528
weighted avg       0.95      0.95      0.95      1528

=== All AUC Scores ===
[0.88908112 0.90580546 0.8935025  0.96539792 0.98269896 0.985198
 0.95482507 0.98212226 0.98788927 0.98356436]


=== Mean AUC Score ===
Mean AUC Score - svc:  0.9530084925447563


In [168]:
Ypred_test = pd.DataFrame(svc.predict(Xtest),columns=["TARGET"])

Ypred_test.head()

#Ypred_prob=pd.DataFrame(svc.predict_proba(Xtest))
#Ypred_prob.head()



,TARGET
0,1.0
1,0.0
2,0.0
3,0.0
4,0.0


In [169]:
#Hyperparameter tuning for svc
parameters={ 'max_iter': range(10,80,10) } #'decision_function_shape': ['ovo','ovr']
svc = SVC()
svc_random = RandomizedSearchCV(svc,parameters,n_iter=10)
svc_random.fit(X_res, Y_res)
rand_parm_svc=svc_random.best_params_
print(rand_parm_svc)

#contruct svc  using the best parameters
svc = LinearSVC(**rand_parm_svc)
svc.fit(X_res, Y_res)
svc_predict_train = svc.predict(X_res)
print("accuracy Score (training) after hypertuning for svc:{0:6f}".format(svc.score(X_res, Y_res)))
print("Confusion Matrix after hypertuning for svc:")
print(confusion_matrix(Y_res,svc_predict_train))
print("=== Classification Report after hypertuning for svc:===")
print(classification_report(Y_res,svc_predict_train))

#run cross-validation on best parameters, get auc score
svc_cv_score = cross_val_score(svc, X_res, Y_res, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(svc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - svc: ",svc_cv_score.mean())

{'max_iter': 70}
accuracy Score (training) after hypertuning for svc:0.768979
Confusion Matrix after hypertuning for svc:
[[855 164]
 [189 320]]
=== Classification Report after hypertuning for svc:===
              precision    recall  f1-score   support

         0.0       0.82      0.84      0.83      1019
         1.0       0.66      0.63      0.64       509

    accuracy                           0.77      1528
   macro avg       0.74      0.73      0.74      1528
weighted avg       0.77      0.77      0.77      1528

=== All AUC Scores ===
[0.74452134 0.77124183 0.79027297 0.83525567 0.82237601 0.89388697
 0.85159554 0.8775471  0.87351019 0.87386139]


=== Mean AUC Score ===
Mean AUC Score - svc:  0.8334068998595361


In [170]:
Ypred_test = pd.DataFrame(svc.predict(Xtest),columns=["TARGET"])

Ypred_test.head()

#Ypred_prob=pd.DataFrame(svc.predict_proba(Xtest))
#Ypred_prob.head()

,TARGET
0,1.0
1,0.0
2,0.0
3,0.0
4,0.0


In [171]:
#KNN=============================================================
#Default mode
knn = KNeighborsClassifier()
knn.fit( X_res, Y_res)
knn_predict_train=knn.predict(X_res)
print("accuracy Score (training) for knn:{0:6f}".format(knn.score( X_res, Y_res)))
print("Confusion Matrix for knn:")
print(confusion_matrix(Y_res,knn_predict_train))
print("=== Classification Report for knn:===")
print(classification_report(Y_res,knn_predict_train))
#run cross-validation on best parameters, get auc score
knn_cv_score = cross_val_score(knn, X_res, Y_res, cv=8, scoring="roc_auc")
print("=== All AUC Scores ===")
print(knn_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - svc: ",knn_cv_score.mean())

accuracy Score (training) for knn:0.918194
Confusion Matrix for knn:
[[927  92]
 [ 33 476]]
=== Classification Report for knn:===
              precision    recall  f1-score   support

         0.0       0.97      0.91      0.94      1019
         1.0       0.84      0.94      0.88       509

    accuracy                           0.92      1528
   macro avg       0.90      0.92      0.91      1528
weighted avg       0.92      0.92      0.92      1528

=== All AUC Scores ===
[0.84313965 0.87402344 0.90679932 0.95250984 0.93227116 0.90657418
 0.9503812  0.95850519]


=== Mean AUC Score ===
Mean AUC Score - svc:  0.9155254967137286


In [172]:
Ypred_test = pd.DataFrame(knn.predict(Xtest),columns=["TARGET"])

Ypred_test.head()

Ypred_prob=pd.DataFrame(knn.predict_proba(Xtest))
Ypred_prob.head()


,0,1
0,0.2,0.8
1,0.2,0.8
2,0.8,0.2
3,1.0,0.0
4,0.4,0.6


In [173]:
#Hyperparameter tuning for knn
parameters={ 'n_neighbors': [3,5,7], 'leaf_size': [20,30,40]}
knn_random = RandomizedSearchCV(knn,parameters,n_iter=10)
knn_random.fit(X_res, Y_res)
rand_parm_knn=knn_random.best_params_
print(rand_parm_knn)

#contruct knn  using the best parameters
knn = KNeighborsClassifier(**rand_parm_knn)
knn.fit(X_res, Y_res)
knn_predict_train = knn.predict(X_res)
print("accuracy Score (training) after hypertuning for knn:{0:6f}".format(knn.score(X_res, Y_res)))
print("Confusion Matrix after hypertuning for knn:")
print(confusion_matrix(Y_res,knn_predict_train))
print("=== Classification Report after hypertuning for knn:===")
print(classification_report(Y_res,knn_predict_train))

#run cross-validation on best parameters, get auc score
knn_cv_score = cross_val_score(knn, X_res, Y_res, cv=8, scoring="roc_auc")
print("=== All AUC Scores ===")
print(knn_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - svc: ",knn_cv_score.mean())

{'n_neighbors': 3, 'leaf_size': 20}
accuracy Score (training) after hypertuning for knn:0.946990
Confusion Matrix after hypertuning for knn:
[[952  67]
 [ 14 495]]
=== Classification Report after hypertuning for knn:===
              precision    recall  f1-score   support

         0.0       0.99      0.93      0.96      1019
         1.0       0.88      0.97      0.92       509

    accuracy                           0.95      1528
   macro avg       0.93      0.95      0.94      1528
weighted avg       0.95      0.95      0.95      1528

=== All AUC Scores ===
[0.88867188 0.89709473 0.93518066 0.97084154 0.94986467 0.92257218
 0.97100362 0.97537808]


=== Mean AUC Score ===
Mean AUC Score - svc:  0.9388259183971632


In [174]:
Ypred_test = pd.DataFrame(knn.predict(Xtest),columns=["TARGET"])

Ypred_test.head()

Ypred_prob=pd.DataFrame(knn.predict_proba(Xtest))
Ypred_prob.head()


,0,1
0,0.000000,1.000000
1,0.333333,0.666667
2,1.000000,0.000000
3,1.000000,0.000000
4,0.000000,1.000000


In [175]:
#STACKING MODELS =====================================================================
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using support vector machine, RandomForest and Decision Tree Classifier\n")

models = [SVC(), RandomForestClassifier(), DecisionTreeClassifier() ]#we r giving models as arguments we can also give those parameters like grid_params
      #we are giving  data after smote and we did smote only for train test
Strain, Stest = stacking(models,                   
                           X_res, Y_res, Xtest,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

___________________________________________________________________________________________
Ensemble Methods Predictions using Neural networks,support vector machine, knn, RandomForest and Decision Tree Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [SVC]
    fold  0:  [0.90861619]
    fold  1:  [0.87172775]
    fold  2:  [0.88219895]
    fold  3:  [0.87139108]
    ----
    MEAN:     [0.88348349] + [0.01514700]
    FULL:     [0.88350785]

model  1:     [RandomForestClassifier]
    fold  0:  [0.92689295]
    fold  1:  [0.92146597]
    fold  2:  [0.91099476]
    fold  3:  [0.90026247]
    ----
    MEAN:     [0.91490404] + [0.01020357]
    FULL:     [0.91492147]

model  2:     [DecisionTreeClassifier]
    fold  0:  [0.90078329]
    fold  1:  [0.89005236]
    fold  2:  [0.89528796]
    fold  3:  [0.89501312]
    ----
    MEAN:     [0.89528418] + [0.00379755]
    FULL:     [0.89528796]

In [176]:
#STACKING - CONTRUCT A decision tree classifier MODEL==============================
model = DecisionTreeClassifier()
    
model = model.fit(Strain, Y_res)
Ypred_train = model.predict(Strain)
print('Final prediction score for ensemble methods using decision tree classifier: [%.8f]' % accuracy_score(Y_res, Ypred_train))
print("Confusion Matrix after STACKING for decision tree classifier :")
print(confusion_matrix(Y_res, Ypred_train))
print("=== Classification Report after STACKING for decision tree classifier===")
print(classification_report(Y_res, Ypred_train))

#run cross-validation on best hyperparameters, get auc score
clf_cv_score = cross_val_score(model, X_res, Y_res, cv=8, scoring="roc_auc")
print("=== All AUC Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Decision Tree: ",clf_cv_score.mean())

Ypred_test = pd.DataFrame(model.predict(Stest),columns=["TARGET"])

Ypred_test.head()

#we are doing one layer with decision tree classifier

Final prediction score for ensemble methods using decision tree classifier: [0.92539267]
Confusion Matrix after STACKING for decision tree classifier :
[[976  43]
 [ 71 438]]
=== Classification Report after STACKING for decision tree classifier===
              precision    recall  f1-score   support

         0.0       0.93      0.96      0.94      1019
         1.0       0.91      0.86      0.88       509

    accuracy                           0.93      1528
   macro avg       0.92      0.91      0.91      1528
weighted avg       0.92      0.93      0.92      1528

=== All AUC Scores ===
[0.8671875  0.87109375 0.90234375 0.92937992 0.93325541 0.8655793
 0.9328209  0.92500937]


=== Mean AUC Score ===
Mean AUC Score - Decision Tree:  0.903333738556118


,TARGET
0,1.0
1,0.0
2,0.0
3,0.0
4,0.0


In [177]:
Ypred_prob=pd.DataFrame(model.predict_proba(Stest))
Ypred_prob.head()

,0,1
0,0.008955,0.991045
1,0.972067,0.027933
2,0.972067,0.027933
3,0.972067,0.027933
4,0.719512,0.280488


In [178]:
#STACKING - CONTRUCT A random forest classfier MODEL==============================
model = RandomForestClassifier()
    
model = model.fit(Strain, Y_res)
Ypred_train = model.predict(Strain)
print('Final prediction score for ensemble methods using random forest: [%.8f]' % accuracy_score(Y_res, Ypred_train))
print("Confusion Matrix after STACKING for random forest :")
print(confusion_matrix(Y_res, Ypred_train))
print("=== Classification Report after STACKING for random forestr===")
print(classification_report(Y_res, Ypred_train))

#run cross-validation on best parameters, get auc score
rfc_cv_score = cross_val_score(model, X_res, Y_res, cv=8, scoring="roc_auc")
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ",rfc_cv_score.mean())

Ypred_test = pd.DataFrame(model.predict(Stest),columns=["TARGET"])

Ypred_test.head()

#we are doing one layer with random forest classifier

Final prediction score for ensemble methods using random forest: [0.92539267]
Confusion Matrix after STACKING for decision tree classifier :
[[976  43]
 [ 71 438]]
=== Classification Report after STACKING for random forestr===
              precision    recall  f1-score   support

         0.0       0.93      0.96      0.94      1019
         1.0       0.91      0.86      0.88       509

    accuracy                           0.93      1528
   macro avg       0.92      0.91      0.91      1528
weighted avg       0.92      0.93      0.92      1528

=== All AUC Scores ===
[0.91229248 0.95257568 0.95227051 0.98025344 0.98954232 0.95994251
 0.98275216 0.98106487]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.9638367462062847


,TARGET
0,1.0
1,0.0
2,0.0
3,0.0
4,0.0


In [179]:
Ypred_prob=pd.DataFrame(model.predict_proba(Stest))
Ypred_prob.head()

,0,1
0,0.010658,0.989342
1,0.970249,0.029751
2,0.970249,0.029751
3,0.970249,0.029751
4,0.731966,0.268034


In [180]:
#STACKING - CONTRUCT A  SVC MODEL==============================
model = SVC()
    
model = model.fit(Strain, Y_res)
Ypred_train = model.predict(Strain)
print('Final prediction score for ensemble methods using  SVC: [%.8f]' % accuracy_score(Y_res, Ypred_train))
print("Confusion Matrix after STACKING for  SVC :")
print(confusion_matrix(Y_res, Ypred_train))
print("=== Classification Report after STACKING for  SVC ===")
print(classification_report(Y_res, Ypred_train))

svc_cv_score = cross_val_score(model, X_res, Y_res, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(svc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - svc: ",svc_cv_score.mean())

Ypred_test = pd.DataFrame(model.predict(Stest),columns=["TARGET"])

Ypred_test.head()

#we are doing one layer  SVC MOdel
#Ypred_prob=pd.DataFrame(model.predict_proba(Stest))
#Ypred_prob.head()


Final prediction score for ensemble methods using linear SVC: [0.92408377]
Confusion Matrix after STACKING for linear SVC :
[[967  52]
 [ 64 445]]
=== Classification Report after STACKING for linear SVC ===
              precision    recall  f1-score   support

         0.0       0.94      0.95      0.94      1019
         1.0       0.90      0.87      0.88       509

    accuracy                           0.92      1528
   macro avg       0.92      0.91      0.91      1528
weighted avg       0.92      0.92      0.92      1528

=== All AUC Scores ===
[0.88908112 0.90580546 0.8935025  0.96539792 0.98269896 0.985198
 0.95482507 0.98212226 0.98788927 0.98356436]


=== Mean AUC Score ===
Mean AUC Score - svc:  0.9530084925447563


,TARGET
0,1.0
1,0.0
2,0.0
3,0.0
4,0.0
